In [1]:
from ucimlrepo import fetch_ucirepo 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
df_features = breast_cancer_wisconsin_original.data.features 
df_targets = breast_cancer_wisconsin_original.data.targets 

## Prediction Goal: 
### To create a prediction model to find what patients are more likely to develop malignant breast cancer. (1989 - 1991)

| Variable Name | Role | Type | Description |
|-----------|:-----------:|:----------:|------------:|
| Sample_code_number | ID | Categorical | ID Number |
| Clump_Thickness | Feature | Integer | 1 - 10 |
| Size_Cell_Uniformity | Feature | Integer | 1 - 10 |
| Shape_Cell_Uniformity | Feature | Integer | 1 - 10 |
| Marginal_Adhesion | Feature | Integer | 1 - 10 |
| Single_Epithelial_Cell_Size | Feature |  Integer | 1 - 10|
| Bare_Nuclei | Feature | Integer | 1 - 10 |
| Bland_Chromatin | Feature | Integer | 1 - 10 |
| Normal_Nuclei | Feature | Integer | 1 - 10 |
| Mitoses | Feature | Integer | 1 - 10 |
| Class | Target | Binary | 2 = Benign, 4 = Malignant |


In [2]:
df_targets.value_counts()
# Checking how many benign vs malignant targets there are 

Class
2        458
4        241
Name: count, dtype: int64

In [3]:
# Concatenate features and targets dataframe into one 
df = pd.concat([df_targets, df_features], axis = 1)
df.head(8)

,Class,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
0,2,5,1,1,1,2,1.0,3,1,1
1,2,5,4,4,5,7,10.0,3,2,1
2,2,3,1,1,1,2,2.0,3,1,1
3,2,6,8,8,1,3,4.0,3,7,1
4,2,4,1,1,3,2,1.0,3,1,1
5,4,8,10,10,8,7,10.0,9,7,1
6,2,1,1,1,1,2,10.0,3,1,1
7,2,2,1,2,1,2,1.0,3,1,1


In [4]:
# Check for any null values, and there are so we have to find out what null values there are
df.isnull().values.any()

True

In [5]:
# We know that Bare_nuclei has null values
df.isnull().sum()
# We will impute later after splitting data

Class                           0
Clump_thickness                 0
Uniformity_of_cell_size         0
Uniformity_of_cell_shape        0
Marginal_adhesion               0
Single_epithelial_cell_size     0
Bare_nuclei                    16
Bland_chromatin                 0
Normal_nucleoli                 0
Mitoses                         0
dtype: int64

In [6]:
class_map = {2:1, 4:0}

In [7]:
# Check ratio of benign vs malignant cancer

num_obs = len(df)
benign_num = len(df.loc[df['Class'] == 2])
malig_num = len(df.loc[df['Class'] == 4])

print("Number of Benign Cases: {0} ({1:2.2f}%)".format(benign_num, (benign_num / (benign_num + malig_num)) * 100))
print("Number of Malignant Cases: {0} ({1:2.2f}%)".format(malig_num, (malig_num / (benign_num + malig_num)) * 100))

Number of Benign Cases: 458 (65.52%)
Number of Malignant Cases: 241 (34.48%)


## Splitting the data into Training and Test sets

In [8]:
# Using scikit-learn to split the data

from sklearn.model_selection import train_test_split

feature_column_names = ["Clump_thickness", "Uniformity_of_cell_size", "Uniformity_of_cell_shape", "Marginal_adhesion", "Single_epithelial_cell_size", "Bare_nuclei", "Bland_chromatin", "Normal_nucleoli", "Mitoses"]
predicted_class_name = ["Class"]

X = df.loc[:, feature_column_names]
y = df.loc[:, predicted_class_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 35)

In [9]:
# Checking data splitting

print("{0:0.2f}% of features data in training set".format((len(X_train)/len(df.index))*100))
print("{0:0.2f}% of features data in test set".format((len(X_test)/len(df.index))*100))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

84.98% of features data in training set
15.02% of features data in test set
(594, 9)
(105, 9)
(594, 1)
(105, 1)


## Splitting the Training data into Validation set

In [10]:
# Splitting for a validation set of data

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 35) #test set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.17, random_state = 35) #validation set


#And checking split for training/ test/ validation sets
print("{0:0.2f}% of features data in training set".format((len(X_train)/len(df.index))*100))
print("{0:0.2f}% of features data in test set".format((len(X_test)/len(df.index))*100))
print("{0:0.2f}% of features data in validation set".format((len(X_valid)/len(df.index))*100))


70.53% of features data in training set
15.02% of features data in test set
14.45% of features data in validation set


In [11]:
# Impute missing data in Bare Nuclei
from sklearn.impute import SimpleImputer

bare_nuclei_impute = SimpleImputer(missing_values = np.nan, strategy = "mean")

X_train = bare_nuclei_impute.fit_transform(X_train)
X_test = bare_nuclei_impute.fit_transform(X_test)
X_valid = bare_nuclei_impute.fit_transform(X_valid)

In [12]:
#Check if the percentages of 2 and 4 are similar between sets

print("Original Benign: {0} ({1:0.2f}%)".format(len(df.loc[df["Class"] == 2]), (len(df.loc[df["Class"] == 2])/len(df.index))*100 ))
print("Original Malignant: {0} ({1:0.2f}%)".format(len(df.loc[df['Class'] == 4]), (len(df.loc[df['Class'] == 4])/len(df.index)) * 100))
print("")

print("Training Benign  : {0} ({1:0.2f}%)".format(len(y_train[y_train['Class'] == 2]), (len(y_train[y_train['Class'] == 2])/len(y_train) * 100.0)))
print("Training Malignant : {0} ({1:0.2f}%)".format(len(y_train[y_train['Class'] == 4]), (len(y_train[y_train['Class'] == 4])/len(y_train) * 100.0)))

print("")

print("Test Benign  : {0} ({1:0.2f}%)".format(len(y_test[y_test['Class'] == 2]), (len(y_test[y_test['Class'] == 2])/len(y_test)) * 100.0))
print("Test Malignant : {0} ({1:0.2f}%)".format(len(y_test[y_test['Class'] == 4]), (len(y_test[y_test['Class'] == 4])/len(y_test)) * 100.0))
print("")
print("Valid Benign  : {0} ({1:0.2f}%)".format(len(y_valid[y_valid['Class'] == 2]), (len(y_valid[y_valid['Class'] == 2])/len(y_valid)) * 100.0))
print("Valid Malignant : {0} ({1:0.2f}%)".format(len(y_valid[y_valid['Class'] == 4]), (len(y_valid[y_valid['Class']== 4])/len(y_valid)) * 100.0))

Original Benign: 458 (65.52%)
Original Malignant: 241 (34.48%)

Training Benign  : 314 (63.69%)
Training Malignant : 179 (36.31%)

Test Benign  : 70 (66.67%)
Test Malignant : 35 (33.33%)

Valid Benign  : 74 (73.27%)
Valid Malignant : 27 (26.73%)


# Splitting and Preprocessing data completed (for now)

# Training model on a Logistic Regression algorithm

In [33]:
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics

lrcv_model = LogisticRegressionCV(Cs = 2, fit_intercept = False, random_state = 35, solver = "liblinear", cv = 10, class_weight = 'balanced', n_jobs = -1)



In [34]:
lrcv_model.fit(X_train, y_train.values.ravel()) # Fitting the model according to the given training data (X_train and y_train)
#method.fit() trains the model

#performance on X_train training data
lrcv_predict_train = lrcv_model.predict(X_test)

In [36]:
#Looking at accuracy of the model using y_train
print("Accuracy: {0:.2f}".format(metrics.accuracy_score(y_test, lrcv_predict_train)))


Accuracy: 0.85


In [46]:
from sklearn.metrics import precision_score

con_matrix = metrics.confusion_matrix(y_test, lrcv_predict_train)
print(con_matrix)

#[TP | FP]
#[FN | TN]

class_report = metrics.classification_report(y_test, lrcv_predict_train)
print(class_report)

precision_score(y_test, lrcv_predict_train, average = None)

print("Precision score: {}".format(precision_score(y_test, lrcv_predict_train), pos_label = 2))


[[59 11]
 [ 5 30]]
              precision    recall  f1-score   support

           2       0.92      0.84      0.88        70
           4       0.73      0.86      0.79        35

    accuracy                           0.85       105
   macro avg       0.83      0.85      0.84       105
weighted avg       0.86      0.85      0.85       105



ValueError: pos_label=1 is not a valid label. It should be one of [2, 4]